In [764]:
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from selenium import webdriver

# 1.0 Scraping boxofficemojo.com website

In [55]:
R=requests.get("https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW")
Soup=BeautifulSoup(R.text,'html.parser')
listo5=[]
listo6=[]

In [84]:
#We got all movies in a page
for x in Soup.find_all(href=True):
    if x["href"].startswith('/title/'): # fist try of all links
         listo5.append(x["href"])
    elif x["href"].startswith("/chart/"): # getting one sample page link
          listo6.append(x["href"])
    else:
          continue
# by usig one sample , getting all pages links        
newlinkss=["https://www.boxofficemojo.com/"]
i=200
for _ in range(5):
    newlinkss.append("https://www.boxofficemojo.com/"+listo6[0].split("=")[0]+str(i))
    i+=200
newlinkss
#Realizing there is difference in original links

['https://www.boxofficemojo.com/',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?area200',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?area400',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?area600',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?area800',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?area1000']

In [ ]:
for i, az in enumerate(newlinkss): #i index . az elements
        newlinkss[i]=az.replace("area","offset=") # Correcting the difference 

In [316]:
newlinkss[0]="https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/" #Now we have actual links of pages so we can get all movies' links
del(newlinkss[5])
newlinkss

['https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?offset=200',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?offset=400',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?offset=600',
 'https://www.boxofficemojo.com//chart/ww_top_lifetime_gross/?offset=800']

In [317]:
def allmovies(listos,bookslists):
    for links in listos:
        R=requests.get(links)
        Soup=BeautifulSoup(R.text,'html.parser')
        for x in Soup.find_all(href=True):
            if x["href"].startswith('/title/'):
                bookslists.append(x["href"])
            else:
                  continue
    return bookslists

In [1645]:
blankpage=[]
allmovies(newlinkss,blankpage)
#We have all movie link endings 

In [1662]:
MovieIDs=blankpage.copy()
MovieIDs[0:10]

['tt0499549',
 'tt4154796',
 'tt1630029',
 'tt0120338',
 'tt2488496',
 'tt4154756',
 'tt10872600',
 'tt0369610',
 'tt6105098',
 'tt0848228']

In [1663]:
full_links=blankpage.copy()
for i,az in enumerate(full_links):
     full_links[i]="https://www.boxofficemojo.com/"+az
full_links[0:10]

['https://www.boxofficemojo.com//title/tt0499549/?ref_=bo_cso_table_1',
 'https://www.boxofficemojo.com//title/tt4154796/?ref_=bo_cso_table_2',
 'https://www.boxofficemojo.com//title/tt1630029/?ref_=bo_cso_table_3',
 'https://www.boxofficemojo.com//title/tt0120338/?ref_=bo_cso_table_4',
 'https://www.boxofficemojo.com//title/tt2488496/?ref_=bo_cso_table_5',
 'https://www.boxofficemojo.com//title/tt4154756/?ref_=bo_cso_table_6',
 'https://www.boxofficemojo.com//title/tt10872600/?ref_=bo_cso_table_7',
 'https://www.boxofficemojo.com//title/tt0369610/?ref_=bo_cso_table_8',
 'https://www.boxofficemojo.com//title/tt6105098/?ref_=bo_cso_table_9',
 'https://www.boxofficemojo.com//title/tt0848228/?ref_=bo_cso_table_10']

In [1664]:
for i,az in enumerate(MovieIDs):
    MovieIDs[i]=az.split("/?ref")[0].split("/title/")[1]
MovieIDs[0:10]

['tt0499549',
 'tt4154796',
 'tt1630029',
 'tt0120338',
 'tt2488496',
 'tt4154756',
 'tt10872600',
 'tt0369610',
 'tt6105098',
 'tt0848228']

In [512]:
MyOwnlist=[] 

In [513]:
specs(tqdm(full_links))

100%|███████████████████████████████████████| 1000/1000 [13:23<00:00,  1.24it/s]


In [511]:
def specs(links):
    for link in links:
        Rr=requests.get(link)
        Soup=BeautifulSoup(Rr.text,'html.parser')
        try:
            name=Soup.find_all('h1',{'class':'a-size-extra-large'})[0].text.split("(")[0]
            year=Soup.find_all('h1',{'class':'a-size-extra-large'})[0].text.split("(")[1].replace(")",'')
        except (AttributeError, IndexError):
            pass
        
        ilkin=Soup.find('div',{'class':'a-section a-spacing-none mojo-performance-summary-table'})
        try: 
            domestic=ilkin.find_all('span',{'class':'percent'})[0].text
            foreign=ilkin.find_all('span',{'class':'percent'})[1].text
        except (AttributeError, IndexError):
            pass
        money=Soup.find_all('span',{'class':'money'}) 
        try:
            domesticm=money[0].text
            internationalm=money[1].text
            wordlwidem=money[2].text
        except (AttributeError, IndexError):
            pass
        opwitable=Soup.find("div",{"class":"a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile"})

        A=opwitable.find_all('div',{'class':"a-section a-spacing-none"})
        if len(A)>6:
            try:
                distributor=A[0].find_all('span')[1].text.replace("\n","")
                DomesticOpening=A[1].find_all('span')[1].text.replace("\n","")
                Budget=A[2].find_all('span')[1].text.replace("\n","")
                Date=A[3].find_all('span')[1].text.replace("\n","")
                MPAA=A[4].find_all('span')[1].text.replace("\n","")
                Time=A[5].find_all('span')[1].text.replace("\n","")
                genres=A[6].find_all('span')[1].text.replace("\n","")
            except (AttributeError, IndexError):
                pass
        elif (len(A)>4 & len(A)<6):
            try:
                distributor=A[0].find_all('span')[1].text.replace("\n","")
                DomesticOpening=A[1].find_all('span')[1].text.replace("\n","")
                Budget=A[2].find_all('span')[1].text.replace("\n","")
                Date=A[3].find_all('span')[1].text.replace("\n","")
                MPAA="Unknown Mpaa"
                Time=A[4].find_all('span')[1].text.replace("\n","")
                genres=A[5].find_all('span')[1].text.replace("\n","")
            except (AttributeError, IndexError):
                pass
        else:
            pass
        movieinfo={ "Name":name,
               "Year":year,
               "Domestic Percentage":domestic ,
               "Foreign percentage":foreign,
               "Domestic Money":domesticm,
               "international":internationalm,
               "wordlwide":wordlwidem,
               "distributor":distributor,
               "DomesticOpening":DomesticOpening,
               "Budget":Budget,
               "Date":Date,
               "MPAA":MPAA,
               "Time":Time,
               "genres":genres
        }
        MyOwnlist.append(movieinfo)

In [564]:
Mydf=pd.DataFrame(MyOwnlist)

In [565]:
Mydf

,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,Avatar,2009,26.9%,73.1%,"$785,221,649","$2,138,484,377","$2,923,706,026",Twentieth Century FoxSee full company information,"$77,025,481","$237,000,000","December 16, 2009 (EMEA, LATAM)",PG-13,2 hr 42 min,Action Adventure Fantasy...
1,Avengers: Endgame,2019,30.7%,69.3%,"$858,373,000","$1,941,066,100","$2,799,439,100",Walt Disney Studios Motion PicturesSee full co...,"$357,115,007","$356,000,000","April 24, 2019 (21 markets)",PG-13,3 hr 1 min,Action Adventure Drama ...
2,Avatar: The Way of Water,2022,29.5%,70.5%,"$684,075,767","$1,636,174,514","$2,320,250,281",20th Century StudiosSee full company information,"$134,100,226","December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...,See more details at IMDbPro
3,Titanic,1997,29.8%,70.2%,"$674,292,608","$1,590,450,697","$2,264,743,305",Paramount PicturesSee full company information,"$28,638,131","$200,000,000","December 19, 1997 (Domestic)",PG-13,3 hr 14 min,Drama Romance
4,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,"$936,662,225","$1,134,647,993","$2,071,310,218",Walt Disney Studios Motion PicturesSee full co...,"$247,966,675","$245,000,000","December 16, 2015 (APAC, EMEA)",PG-13,2 hr 18 min,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,The Scorpion King,2002,50.4%,49.6%,"$91,047,077","$89,583,830","$180,630,907",Universal PicturesSee full company information,"$36,075,875","$60,000,000","April 18, 2002 (APAC)",PG-13,1 hr 32 min,Action Adventure Fantasy
996,Sweet Home Alabama,2002,70.4%,29.6%,"$127,223,418","$53,399,006","$180,622,424",Walt Disney Studios Motion PicturesSee full co...,"$35,648,740","$30,000,000","September 27, 2002 (Domestic)",PG-13,1 hr 48 min,Comedy Romance
997,Daddy's Home 2,2017,57.6%,42.4%,"$104,029,443","$76,584,381","$180,613,824",Paramount PicturesSee full company information,"$29,651,193","$69,000,000","November 9, 2017 (Netherlands)",PG-13,1 hr 40 min,Comedy
998,Hacksaw Ridge,2016,37.2%,62.8%,"$67,209,615","$113,354,021","$180,563,636",Lions Gate FilmsSee full company information,"$15,190,758","$40,000,000","November 3, 2016 (EMEA, APAC)",R,2 hr 19 min,Biography Drama History ...


In [1649]:
pd.set_option('display.max_rows',None)
pd.reset_option('display.max_rows')

In [383]:
MovieIDs2=pd.Series(MovieIDs)
MovieIDs2

0      tt0499549
1      tt4154796
2      tt1630029
3      tt0120338
4      tt2488496
         ...    
995    tt0277296
996    tt0256415
997    tt5657846
998    tt2119532
999    tt0453467
Length: 1000, dtype: object

# 1.1 Data Cleaning

In [575]:
#Handling Missing Budget values , not containing $ means there is incorrect value
Filter=Mydf[~Mydf["Budget"].str.strip().str.contains("[$]")]
Filter["genres"]=Filter["Time"]
Filter["Time"]=Filter["MPAA"]
Filter["Budget"]="Unknown"
Filter

/var/folders/bv/bgrpp52x05nf96zx_01sm2_40000gn/T/ipykernel_26775/4010276228.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Filter["Budget"]="Unknown"


,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
2,Avatar: The Way of Water,2022,29.5%,70.5%,"$684,075,767","$1,636,174,514","$2,320,250,281",20th Century StudiosSee full company information,"$134,100,226",Unknown,"December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...
5,Avengers: Infinity War,2018,33.1%,66.9%,"$678,815,482","$1,373,599,557","$2,052,415,039",Walt Disney Studios Motion PicturesSee full co...,"$257,698,183",Unknown,"April 25, 2018 (APAC, EMEA)",PG-13,2 hr 29 min,Action Adventure Sci-Fi
6,Spider-Man: No Way Home,2021,42.4%,57.6%,"$814,115,070","$1,107,732,041","$1,921,847,111",Sony Pictures Entertainment (SPE)See full comp...,"$260,138,569",Unknown,"December 15, 2021 (14 markets)",PG-13,2 hr 28 min,Action Adventure Fantasy...
11,Top Gun: Maverick,2022,48.1%,51.9%,"$718,732,821","$776,963,471","$1,495,696,292",Paramount PicturesSee full company information,"$126,707,459",Unknown,"May 20, 2022 (Iceland)",PG-13,2 hr 10 min,Action Drama
13,Barbie,2023,44.1%,55.9%,"$635,235,535","$804,700,000","$1,439,935,535",Warner Bros.See full company information,"$162,022,044",Unknown,"July 19, 2023 (EMEA, APAC)",PG-13,1 hr 54 min,Adventure Comedy Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,The Man in the Iron Mask,1998,31.1%,68.9%,"$56,968,902","$126,000,000","$182,968,902",Metro-Goldwyn-Mayer (MGM)See full company info...,"$17,271,450",Unknown,"March 13, 1998 (Domestic)",PG-13,2 hr 12 min,Action Adventure Drama
982,Cape Fear,1991,43.4%,56.6%,"$79,091,969","$103,200,000","$182,291,969",Universal PicturesSee full company information,"$10,261,025",Unknown,"November 15, 1991 (Domestic)",R,2 hr 8 min,Crime Thriller
989,The First Wives Club,1996,58.1%,41.9%,"$105,489,203","$76,000,000","$181,489,203",Paramount PicturesSee full company information,"$18,913,411",Unknown,"September 20, 1996 (Domestic)",PG,1 hr 43 min,Comedy
990,Cliff Walkers,2021,–,99.9%,"$152,972","$181,172,593","$181,325,565",CMC PicturesSee full company information,"$53,142",Unknown,"April 29, 2021 (New Zealand)",2 hr,Unknown Mpaa,Crime Drama Thriller


In [577]:
result=Filter.copy()
Mydf[~Mydf["Budget"].str.strip().str.contains("[$]")]=result
Mydf

,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,Avatar,2009,26.9%,73.1%,"$785,221,649","$2,138,484,377","$2,923,706,026",Twentieth Century FoxSee full company information,"$77,025,481","$237,000,000","December 16, 2009 (EMEA, LATAM)",PG-13,2 hr 42 min,Action Adventure Fantasy...
1,Avengers: Endgame,2019,30.7%,69.3%,"$858,373,000","$1,941,066,100","$2,799,439,100",Walt Disney Studios Motion PicturesSee full co...,"$357,115,007","$356,000,000","April 24, 2019 (21 markets)",PG-13,3 hr 1 min,Action Adventure Drama ...
2,Avatar: The Way of Water,2022,29.5%,70.5%,"$684,075,767","$1,636,174,514","$2,320,250,281",20th Century StudiosSee full company information,"$134,100,226",Unknown,"December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...
3,Titanic,1997,29.8%,70.2%,"$674,292,608","$1,590,450,697","$2,264,743,305",Paramount PicturesSee full company information,"$28,638,131","$200,000,000","December 19, 1997 (Domestic)",PG-13,3 hr 14 min,Drama Romance
4,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,"$936,662,225","$1,134,647,993","$2,071,310,218",Walt Disney Studios Motion PicturesSee full co...,"$247,966,675","$245,000,000","December 16, 2015 (APAC, EMEA)",PG-13,2 hr 18 min,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,The Scorpion King,2002,50.4%,49.6%,"$91,047,077","$89,583,830","$180,630,907",Universal PicturesSee full company information,"$36,075,875","$60,000,000","April 18, 2002 (APAC)",PG-13,1 hr 32 min,Action Adventure Fantasy
996,Sweet Home Alabama,2002,70.4%,29.6%,"$127,223,418","$53,399,006","$180,622,424",Walt Disney Studios Motion PicturesSee full co...,"$35,648,740","$30,000,000","September 27, 2002 (Domestic)",PG-13,1 hr 48 min,Comedy Romance
997,Daddy's Home 2,2017,57.6%,42.4%,"$104,029,443","$76,584,381","$180,613,824",Paramount PicturesSee full company information,"$29,651,193","$69,000,000","November 9, 2017 (Netherlands)",PG-13,1 hr 40 min,Comedy
998,Hacksaw Ridge,2016,37.2%,62.8%,"$67,209,615","$113,354,021","$180,563,636",Lions Gate FilmsSee full company information,"$15,190,758","$40,000,000","November 3, 2016 (EMEA, APAC)",R,2 hr 19 min,Biography Drama History ...


In [646]:
Copyasi=Mydf.copy()

In [621]:
Mydf

,ID,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,Avatar,2009,26.9%,73.1%,"$785,221,649","$2,138,484,377","$2,923,706,026",Twentieth Century FoxSee full company information,"$77,025,481","$237,000,000","December 16, 2009 (EMEA, LATAM)",PG-13,2 hr 42 min,Action Adventure Fantasy...
1,Avengers: Endgame,2019,30.7%,69.3%,"$858,373,000","$1,941,066,100","$2,799,439,100",Walt Disney Studios Motion PicturesSee full co...,"$357,115,007","$356,000,000","April 24, 2019 (21 markets)",PG-13,3 hr 1 min,Action Adventure Drama ...
2,Avatar: The Way of Water,2022,29.5%,70.5%,"$684,075,767","$1,636,174,514","$2,320,250,281",20th Century StudiosSee full company information,"$134,100,226",Unknown,"December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...
3,Titanic,1997,29.8%,70.2%,"$674,292,608","$1,590,450,697","$2,264,743,305",Paramount PicturesSee full company information,"$28,638,131","$200,000,000","December 19, 1997 (Domestic)",PG-13,3 hr 14 min,Drama Romance
4,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,"$936,662,225","$1,134,647,993","$2,071,310,218",Walt Disney Studios Motion PicturesSee full co...,"$247,966,675","$245,000,000","December 16, 2015 (APAC, EMEA)",PG-13,2 hr 18 min,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,The Scorpion King,2002,50.4%,49.6%,"$91,047,077","$89,583,830","$180,630,907",Universal PicturesSee full company information,"$36,075,875","$60,000,000","April 18, 2002 (APAC)",PG-13,1 hr 32 min,Action Adventure Fantasy
996,Sweet Home Alabama,2002,70.4%,29.6%,"$127,223,418","$53,399,006","$180,622,424",Walt Disney Studios Motion PicturesSee full co...,"$35,648,740","$30,000,000","September 27, 2002 (Domestic)",PG-13,1 hr 48 min,Comedy Romance
997,Daddy's Home 2,2017,57.6%,42.4%,"$104,029,443","$76,584,381","$180,613,824",Paramount PicturesSee full company information,"$29,651,193","$69,000,000","November 9, 2017 (Netherlands)",PG-13,1 hr 40 min,Comedy
998,Hacksaw Ridge,2016,37.2%,62.8%,"$67,209,615","$113,354,021","$180,563,636",Lions Gate FilmsSee full company information,"$15,190,758","$40,000,000","November 3, 2016 (EMEA, APAC)",R,2 hr 19 min,Biography Drama History ...


In [1665]:
#If column contains see more details at IMDBPro , it means we have right value otherwise it means there is some incorrect value
filter2=Mydf[Mydf["genres"]=="See more details at IMDbPro"]
filter2["genres"]=filter2['Time']
filter2["Time"]=filter2["MPAA"]
filter2["MPAA"]="Unknown"
Mydf[Mydf["genres"]=="See more details at IMDbPro"]=filter2
filter2

,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
39,Toy Story 3,2010,38.9%,61.1%,"$415,004,880","$652,311,221","$1,067,316,101",Walt Disney Studios Motion PicturesSee full co...,"$110,307,189","$200,000,000","June 16, 2010 (Egypt)",Unknown,1 hr 43 min,Adventure Animation Come...
65,Finding Nemo,2003,40.4%,59.6%,"$380,843,261","$560,794,699","$941,637,960",Walt Disney Studios Motion PicturesSee full co...,"$70,251,710","$94,000,000","May 30, 2003 (Domestic)",Unknown,1 hr 40 min,Adventure Animation Come...
81,Wolf Warrior 2,2017,0.3%,99.7%,"$2,721,100","$867,604,339","$870,325,439",The H CollectiveSee full company information,"$219,022","$30,100,000","July 27, 2017 (China, APAC)",Unknown,2 hr 3 min,Action Adventure Drama ...
116,Fast & Furious Presents: Hobbs & Shaw,2019,22.9%,77.1%,"$173,956,935","$586,775,991","$760,732,926",Universal PicturesSee full company information,"$60,038,950","$200,000,000","July 1, 2019 (Portugal)",Unknown,2 hr 17 min,Action Adventure Thriller
229,WALL·E,2008,42.9%,57.1%,"$223,808,164","$297,503,726","$521,311,890",Walt Disney Studios Motion PicturesSee full co...,"$63,087,526","$180,000,000","June 26, 2008 (LATAM)",Unknown,1 hr 38 min,Adventure Animation Fami...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,Rambo III,1988,28.4%,71.6%,"$53,715,611","$135,300,000","$189,015,611",TriStar PicturesSee full company information,"$13,034,238","$63,000,000","May 25, 1988 (Domestic)",Unknown,1 hr 42 min,Action Adventure Thriller
957,The Jungle Book 2,2003,25.7%,74.3%,"$47,901,582","$138,402,177","$186,303,759",Walt Disney Studios Motion PicturesSee full co...,"$11,441,733","$20,000,000","February 5, 2003 (France)",Unknown,1 hr 12 min,Adventure Animation Come...
974,102 Dalmatians,2000,36.5%,63.5%,"$66,957,026","$116,654,745","$183,611,771",Walt Disney Studios Motion PicturesSee full co...,"$19,883,351","$85,000,000","November 22, 2000 (Domestic)",Unknown,1 hr 40 min,Adventure Comedy Family
975,Stand by Me Doraemon,2014,–,100%,"$183,442,714","$183,442,714","$1,370,376",Walt Disney Studios Motion PicturesSee full co...,"$19,883,351","$85,000,000","November 22, 2000 (Domestic)",Unknown,1 hr 40 min,Adventure Comedy Family


In [644]:
#Editing some incorrect values as unkown so I can acces them altogether as they have same value ."Unknown"
Mydf[Mydf["Time"].str.contains("Unknown Mpaa")]="Unknown"
Mydf.loc[Mydf["MPAA"].str.contains("hr"), "MPAA"] = "Unknown"
Mydf["MPAA"].value_counts()

PG-13       451
PG          218
R           201
Unknown     114
G            15
Approved      1
Name: MPAA, dtype: int64

In [627]:
#Changing first column name
names=Mydf.columns
names=list(names)
names[0]="ID"
Mydf.columns=names
Mydf.iloc[65:73,:]

,ID,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
65,tt0266543,Finding Nemo,2003,40.4%,59.6%,"$380,843,261","$560,794,699","$941,637,960",Walt Disney Studios Motion PicturesSee full co...,"$70,251,710","$94,000,000","May 30, 2003 (Domestic)",Unknown,1 hr 40 min,Adventure Animation Come...
66,tt5113044,Minions: The Rise of Gru,2022,39.3%,60.7%,"$369,695,210","$569,933,000","$939,628,210",Universal PicturesSee full company information,"$107,010,140",Unknown,"June 23, 2022 (Australia)",PG,1 hr 27 min,Adventure Animation Come...
67,tt0417741,Harry Potter and the Half-Blood Prince,2009,32.4%,67.6%,"$302,334,374","$632,185,012","$934,519,387",Warner Bros.See full company information,"$77,835,727","$250,000,000","July 15, 2009 (20 markets)",PG,2 hr 33 min,Action Adventure Family ...
68,tt0298148,Shrek 2,2004,47.5%,52.5%,"$441,226,247","$487,534,523","$928,760,770",DreamWorks DistributionSee full company inform...,"$108,037,878","$150,000,000","May 19, 2004 (Domestic)",PG,1 hr 33 min,Adventure Animation Come...
69,tt0295297,Harry Potter and the Chamber of Secrets,2002,28.4%,71.6%,"$262,641,637","$663,560,091","$926,201,729",Warner Bros.See full company information,"$88,357,488","$100,000,000","November 14, 2002 (EMEA)",PG,2 hr 41 min,Adventure Family Fantasy...
70,tt1727824,Bohemian Rhapsody,2018,23.8%,76.2%,"$216,668,042","$694,141,269","$910,809,311",Twentieth Century FoxSee full company information,"$51,061,119","$52,000,000","October 26, 2018 (United Kingdom)",PG-13,2 hr 14 min,Biography Drama Music
71,tt13462900,The Battle at Lake Changjin,2021,–,100%,"$342,411","$902,206,065","$902,548,476",CMC PicturesSee full company information,"$105,768",Unknown,"October 1, 2021 (China)",2 hr 56 min,Unknown Mpaa,Action Drama History ...
72,tt0120737,The Lord of the Rings: The Fellowship of the R...,2001,35.2%,64.8%,"$316,115,420","$582,386,399","$898,501,819",New Line CinemaSee full company information,"$47,211,490","$93,000,000","December 19, 2001 (16 markets)",PG-13,2 hr 58 min,Action Adventure Drama ...


In [1640]:
""""We have the main template""" 

'"We have the main template'

In [645]:
Mydf.to_excel("FirstDraft.xlsx") #continuing analysis in Excel

In [1063]:
#We realize some movies have 2 different versions
Mydf=pd.concat([MovieIDs2,Mydf],axis=1)
Mydf["Name"].value_counts().head(15)

The Little Mermaid               2
Beauty and the Beast             2
Total Recall                     2
Hercules                         2
The Mummy                        2
Ghostbusters                     2
The Jungle Book                  2
Aladdin                          2
Godzilla                         2
The Addams Family                2
The Lion King                    2
Teenage Mutant Ninja Turtles     2
Get Smart                        1
The Bad Guys                     1
Dark Phoenix                     1
Name: Name, dtype: int64

# 2.0 Scraping Second Website watch.plex.tv

In [1107]:
Mydf

,ID,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,tt0499549,Avatar,2009,26.9%,73.1%,"$785,221,649","$2,138,484,377","$2,923,706,026",Twentieth Century FoxSee full company information,"$77,025,481","$237,000,000","December 16, 2009 (EMEA, LATAM)",PG-13,2 hr 42 min,Action Adventure Fantasy...
1,tt4154796,Avengers: Endgame,2019,30.7%,69.3%,"$858,373,000","$1,941,066,100","$2,799,439,100",Walt Disney Studios Motion PicturesSee full co...,"$357,115,007","$356,000,000","April 24, 2019 (21 markets)",PG-13,3 hr 1 min,Action Adventure Drama ...
2,tt1630029,Avatar: The Way of Water,2022,29.5%,70.5%,"$684,075,767","$1,636,174,514","$2,320,250,281",20th Century StudiosSee full company information,"$134,100,226",Unknown,"December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...
3,tt0120338,Titanic,1997,29.8%,70.2%,"$674,292,608","$1,590,450,697","$2,264,743,305",Paramount PicturesSee full company information,"$28,638,131","$200,000,000","December 19, 1997 (Domestic)",PG-13,3 hr 14 min,Drama Romance
4,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,"$936,662,225","$1,134,647,993","$2,071,310,218",Walt Disney Studios Motion PicturesSee full co...,"$247,966,675","$245,000,000","December 16, 2015 (APAC, EMEA)",PG-13,2 hr 18 min,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,2002,50.4%,49.6%,"$91,047,077","$89,583,830","$180,630,907",Universal PicturesSee full company information,"$36,075,875","$60,000,000","April 18, 2002 (APAC)",PG-13,1 hr 32 min,Action Adventure Fantasy
996,tt0256415,Sweet Home Alabama,2002,70.4%,29.6%,"$127,223,418","$53,399,006","$180,622,424",Walt Disney Studios Motion PicturesSee full co...,"$35,648,740","$30,000,000","September 27, 2002 (Domestic)",PG-13,1 hr 48 min,Comedy Romance
997,tt5657846,Daddy's Home 2,2017,57.6%,42.4%,"$104,029,443","$76,584,381","$180,613,824",Paramount PicturesSee full company information,"$29,651,193","$69,000,000","November 9, 2017 (Netherlands)",PG-13,1 hr 40 min,Comedy
998,tt2119532,Hacksaw Ridge,2016,37.2%,62.8%,"$67,209,615","$113,354,021","$180,563,636",Lions Gate FilmsSee full company information,"$15,190,758","$40,000,000","November 3, 2016 (EMEA, APAC)",R,2 hr 19 min,Biography Drama History ...


In [1668]:
Listlink=Mydf["Name"]
Listlink=list(Listlink)

In [1669]:
# deleting all unusual characters as we will proceed with names for scraping another website
for i in range(len(Listlink)):
    Listlink[i]=Listlink[i].strip().replace(" ","-").replace(".","").replace("'","").replace(":","").replace("&","and").replace(",","").replace("II","ii").replace("I",'i').replace("--","-").lower()

In [1670]:
for a in range(len(Listlink)):
    Listlink[a]="https://watch.plex.tv/movie/"+Listlink[a]

In [829]:
Mysecondlist=[]

In [1030]:
mylingo
mydingo
Listlink=pd.DataFrame(Listlink)
Listlink.to_csv("AllSql.csv")
Listlink[0].value_counts().head(50)
#Moving to sql

https://watch.plex.tv/movie/the-little-mermaid                            2
https://watch.plex.tv/movie/beauty-and-the-beast                          2
https://watch.plex.tv/movie/total-recall                                  2
https://watch.plex.tv/movie/hercules                                      2
https://watch.plex.tv/movie/the-mummy                                     2
https://watch.plex.tv/movie/ghostbusters                                  2
https://watch.plex.tv/movie/the-jungle-book                               2
https://watch.plex.tv/movie/aladdin                                       2
https://watch.plex.tv/movie/godzilla                                      2
https://watch.plex.tv/movie/the-addams-family                             2
https://watch.plex.tv/movie/the-lion-king                                 2
https://watch.plex.tv/movie/teenage-mutant-ninja-turtles                  2
https://watch.plex.tv/movie/get-smart                                     1
https://watc

In [1671]:
"""I am getting problem as I have some duplicate links for same films different versions.I got a file with corrected links for different versions and will utilize Conditional joining to join them to my link list in sql.There is no dfd value in that data , I just used this row  as a sign to show I am moving on SQL. dfd.to_csv("RepairSql.csv") """

'I am getting problem as I have some duplicate links for same films different versions.I got a file with corrected links for different versions and will utilize Conditional joining to join them to my link list in sql.There is no dfd value in that data , I just used this row  as a sign to show I am moving on SQL. dfd.to_csv("RepairSql.csv") '

In [1015]:
#All links corrected and ready 
alllinks=pd.read_csv("/Users/aa085/Lets begin/Linksready.csv")
alllinks

,numer,linkselection
0,0,linksss
1,0,linksss
2,1,https://watch.plex.tv/movie/avengers-endgame
3,2,https://watch.plex.tv/movie/avatar-2
4,3,https://watch.plex.tv/movie/titanic
...,...,...
996,995,https://watch.plex.tv/movie/the-scorpion-king
997,996,https://watch.plex.tv/movie/sweet-home-alabama
998,997,https://watch.plex.tv/movie/daddys-home-2
999,998,https://watch.plex.tv/movie/hacksaw-ridge


In [1108]:
kopyasi=alllinks.copy()

In [1020]:
# Reseting the index
alllinks = alllinks.reset_index(drop=True)
alllinks

,numer,linkselection
0,0,linksss
1,1,https://watch.plex.tv/movie/avengers-endgame
2,2,https://watch.plex.tv/movie/avatar-2
3,3,https://watch.plex.tv/movie/titanic
4,4,https://watch.plex.tv/movie/star-wars-episode-...
...,...,...
995,995,https://watch.plex.tv/movie/the-scorpion-king
996,996,https://watch.plex.tv/movie/sweet-home-alabama
997,997,https://watch.plex.tv/movie/daddys-home-2
998,998,https://watch.plex.tv/movie/hacksaw-ridge


In [1022]:
alllinks.iloc[0,1]="https://watch.plex.tv/movie/avatar"
alllinks

,numer,linkselection
0,0,https://watch.plex.tv/movie/avatar
1,1,https://watch.plex.tv/movie/avengers-endgame
2,2,https://watch.plex.tv/movie/avatar-2
3,3,https://watch.plex.tv/movie/titanic
4,4,https://watch.plex.tv/movie/star-wars-episode-...
...,...,...
995,995,https://watch.plex.tv/movie/the-scorpion-king
996,996,https://watch.plex.tv/movie/sweet-home-alabama
997,997,https://watch.plex.tv/movie/daddys-home-2
998,998,https://watch.plex.tv/movie/hacksaw-ridge


In [1152]:
#Copying Last version of links
lastversion=alllinks["linkselection"]
lastversion=list(lastversion)
len(lastversion)

1000

In [1153]:
Mysecondlist=[]

In [1154]:
def scrapimdb(links):
    for link in links:
        R=requests.get(link)
        Soup=BeautifulSoup(R.text,"html.parser")
        try:
            opwi=Soup.find("div",{"class":"_1duebfh12 _1duebfh9 _1duebfh32 _1duebfh1m _1duebfh2m _1duebfh2i _1duebfh22 _1duebfh4e _1duebfh3e"}).text
            directror=Soup.find("span",{"class":"ineka90 ineka9u ineka9a _1duebfh2m _1duebfh2i _1duebfhfy"}).text
            Name=Soup.find("div",{"class":"_1duebfh12 _1duebfh9 _1duebfh32 _1duebfh2z _1duebfh1q _1duebfh2m _1duebfh2i _1duebfh2a _1duebfh3e _1duebfh3f"}).text
        except (AttributeError, IndexError):
                pass
        movieinfo={
            "Name":Name,
            "All ranks":opwi ,
            "Director":directror
        }
        Mysecondlist.append(movieinfo)

In [1026]:
#Checking last time
alllinks["linkselection"].value_counts().head(10)

https://watch.plex.tv/movie/teenage-mutant-ninja-turtles    2
https://watch.plex.tv/movie/aladdin                         2
https://watch.plex.tv/movie/beauty-and-the-beast            2
https://watch.plex.tv/movie/the-little-mermaid              2
https://watch.plex.tv/movie/godzilla                        2
https://watch.plex.tv/movie/ghostbusters                    2
https://watch.plex.tv/movie/the-ring                        1
https://watch.plex.tv/movie/the-bad-guys                    1
https://watch.plex.tv/movie/the-mask-of-zorro               1
https://watch.plex.tv/movie/the-godfather                   1
Name: linkselection, dtype: int64

In [1155]:
scrapimdb(tqdm(lastversion))

100%|███████████████████████████████████████| 1000/1000 [11:48<00:00,  1.41it/s]


In [1156]:
newdf=pd.DataFrame(Mysecondlist)
newdf

,Name,All ranks,Director
0,Avatar,7.981%82%76%,James Cameron
1,Avengers: Endgame,8.494%90%83%,"Anthony Russo, Joe Russo"
2,Avatar: The Way of Water,7.676%92%77%,James Cameron
3,Titanic,6.160%57%62%,"Werner Klingler, Herbert Selpin"
4,Star Wars: Episode VII - The Force Awakens,7.893%85%73%,J.J. Abrams
...,...,...,...
995,The Scorpion King,5.541%38%56%,Chuck Russell
996,Sweet Home Alabama,6.238%78%64%,Andy Tennant
997,Daddy's Home 2,6.021%51%63%,Sean Anders
998,Hacksaw Ridge,8.184%91%82%,Mel Gibson


# 2.1 Cleaning & Processing second Dataframe

In [1157]:
newdf["Name"].value_counts()

The Little Mermaid             2
Beauty and the Beast           2
Total Recall                   2
Hercules                       2
The Mummy                      2
                              ..
Mamma Mia! Here We Go Again    1
Eternals                       1
Dune                           1
TRON: Legacy                   1
Deja Vu                        1
Name: Name, Length: 986, dtype: int64

In [1164]:
#Adding links to dataframe 
newdf = newdf.drop("Name", axis=1) #deleting duplicated column
newdf = pd.concat([newdf, alllinks["linkselection"]], axis=1) 

In [1673]:
mylist=newdf["All ranks"]
mylist=list(mylist)
IMDB=[]
for a in range(len(mylist)):
     IMDB.append(mylist[a][0:3])
IMDB[:10]

['7.9', '8.4', '7.6', '6.1', '7.8', '8.4', '8.2', '6.9', '6.8', '8.0']

In [1171]:
newdf["IMDB"]=IMDB
newdf

,Name,All ranks,Director,linkselection,IMDB
0,Avatar,7.981%82%76%,James Cameron,https://watch.plex.tv/movie/avatar,7.9
1,Avengers: Endgame,8.494%90%83%,"Anthony Russo, Joe Russo",https://watch.plex.tv/movie/avengers-endgame,8.4
2,Avatar: The Way of Water,7.676%92%77%,James Cameron,https://watch.plex.tv/movie/avatar-2,7.6
3,Titanic,6.160%57%62%,"Werner Klingler, Herbert Selpin",https://watch.plex.tv/movie/titanic,6.1
4,Star Wars: Episode VII - The Force Awakens,7.893%85%73%,J.J. Abrams,https://watch.plex.tv/movie/star-wars-episode-...,7.8
...,...,...,...,...,...
995,The Scorpion King,5.541%38%56%,Chuck Russell,https://watch.plex.tv/movie/the-scorpion-king,5.5
996,Sweet Home Alabama,6.238%78%64%,Andy Tennant,https://watch.plex.tv/movie/sweet-home-alabama,6.2
997,Daddy's Home 2,6.021%51%63%,Sean Anders,https://watch.plex.tv/movie/daddys-home-2,6.0
998,Hacksaw Ridge,8.184%91%82%,Mel Gibson,https://watch.plex.tv/movie/hacksaw-ridge,8.1


In [1173]:
mylist = [sublist[3:] for sublist in mylist]

In [1674]:
Tomatocritical=[]
Tomatoaudience=[]
TQDM=[]

for a in mylist:
    if a.count("%")==3:
        parts = a.split("%")
        Tomatocritical.append(parts[0])
        Tomatoaudience.append(parts[1])
        TQDM.append(parts[2])
    else:
        Tomatocritical.append(np.nan)
        Tomatoaudience.append(np.nan)
        TQDM.append(np.nan)
Tomatocritical
Tomatoaudience

In [1190]:
#Splitting the outputs from scraping
len(TQDM)
len(Tomatocritical)
len(Tomatoaudience)

1000

In [1191]:
newdf["Tomatometer"]=Tomatocritical
newdf["Rotten Tomatos Audience"]=Tomatoaudience
newdf["TMDB"]=TQDM
newdf

,Name,All ranks,Director,linkselection,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB
0,Avatar,7.981%82%76%,James Cameron,https://watch.plex.tv/movie/avatar,7.9,81,82,76
1,Avengers: Endgame,8.494%90%83%,"Anthony Russo, Joe Russo",https://watch.plex.tv/movie/avengers-endgame,8.4,94,90,83
2,Avatar: The Way of Water,7.676%92%77%,James Cameron,https://watch.plex.tv/movie/avatar-2,7.6,76,92,77
3,Titanic,6.160%57%62%,"Werner Klingler, Herbert Selpin",https://watch.plex.tv/movie/titanic,6.1,60,57,62
4,Star Wars: Episode VII - The Force Awakens,7.893%85%73%,J.J. Abrams,https://watch.plex.tv/movie/star-wars-episode-...,7.8,93,85,73
...,...,...,...,...,...,...,...,...
995,The Scorpion King,5.541%38%56%,Chuck Russell,https://watch.plex.tv/movie/the-scorpion-king,5.5,41,38,56
996,Sweet Home Alabama,6.238%78%64%,Andy Tennant,https://watch.plex.tv/movie/sweet-home-alabama,6.2,38,78,64
997,Daddy's Home 2,6.021%51%63%,Sean Anders,https://watch.plex.tv/movie/daddys-home-2,6.0,21,51,63
998,Hacksaw Ridge,8.184%91%82%,Mel Gibson,https://watch.plex.tv/movie/hacksaw-ridge,8.1,84,91,82


In [1226]:
newdf2.to_excel("fixinglinks.xlsx") #further analysis on excel

In [1206]:
newdf2=pd.read_excel("/Users/aa085/Lets begin/fixinglinks.xlsx") #Coming back to python
newdf2=newdf2.drop(["Unnamed: 0"],axis=1)
newdf2.isna().sum()

Name                        0
All ranks                   0
Director                    0
linkselection               0
IMDB                        0
Tomatometer                71
Rotten Tomatos Audience    25
TMDB                        4
dtype: int64

In [1209]:
newdf2=newdf2.drop(["All ranks"],axis=1)

In [1210]:
newdf2 # Ratings ready

,Name,Director,linkselection,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB
0,Avatar,James Cameron,https://watch.plex.tv/movie/avatar,7.9,81.0,82.0,76.0
1,Avengers: Endgame,"Anthony Russo, Joe Russo",https://watch.plex.tv/movie/avengers-endgame,8.4,94.0,90.0,83.0
2,Avatar: The Way of Water,James Cameron,https://watch.plex.tv/movie/avatar-2,7.6,76.0,92.0,77.0
3,Titanic,"Werner Klingler, Herbert Selpin",https://watch.plex.tv/movie/titanic,6.1,60.0,57.0,62.0
4,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,https://watch.plex.tv/movie/star-wars-episode-...,7.8,93.0,85.0,73.0
...,...,...,...,...,...,...,...
995,The Scorpion King,Chuck Russell,https://watch.plex.tv/movie/the-scorpion-king,5.5,41.0,38.0,56.0
996,Sweet Home Alabama,Andy Tennant,https://watch.plex.tv/movie/sweet-home-alabama,6.2,38.0,78.0,64.0
997,Daddy's Home 2,Sean Anders,https://watch.plex.tv/movie/daddys-home-2,6.0,21.0,51.0,63.0
998,Hacksaw Ridge,Mel Gibson,https://watch.plex.tv/movie/hacksaw-ridge,8.1,84.0,91.0,82.0


In [1222]:
Namelist=["Name","IMDB","Tomatometer","Rotten Tomatos Audience","TMDB","Director","linkselection"]
newdf2=newdf2.reindex(Namelist,axis=1)
newdf2

,Name,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,Director,linkselection
0,Avatar,7.9,81.0,82.0,76.0,James Cameron,https://watch.plex.tv/movie/avatar
1,Avengers: Endgame,8.4,94.0,90.0,83.0,"Anthony Russo, Joe Russo",https://watch.plex.tv/movie/avengers-endgame
2,Avatar: The Way of Water,7.6,76.0,92.0,77.0,James Cameron,https://watch.plex.tv/movie/avatar-2
3,Titanic,6.1,60.0,57.0,62.0,"Werner Klingler, Herbert Selpin",https://watch.plex.tv/movie/titanic
4,Star Wars: Episode VII - The Force Awakens,7.8,93.0,85.0,73.0,J.J. Abrams,https://watch.plex.tv/movie/star-wars-episode-...
...,...,...,...,...,...,...,...
995,The Scorpion King,5.5,41.0,38.0,56.0,Chuck Russell,https://watch.plex.tv/movie/the-scorpion-king
996,Sweet Home Alabama,6.2,38.0,78.0,64.0,Andy Tennant,https://watch.plex.tv/movie/sweet-home-alabama
997,Daddy's Home 2,6.0,21.0,51.0,63.0,Sean Anders,https://watch.plex.tv/movie/daddys-home-2
998,Hacksaw Ridge,8.1,84.0,91.0,82.0,Mel Gibson,https://watch.plex.tv/movie/hacksaw-ridge


In [1223]:
newdf2 #READYTOUSE

,Name,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,Director,linkselection
0,Avatar,7.9,81.0,82.0,76.0,James Cameron,https://watch.plex.tv/movie/avatar
1,Avengers: Endgame,8.4,94.0,90.0,83.0,"Anthony Russo, Joe Russo",https://watch.plex.tv/movie/avengers-endgame
2,Avatar: The Way of Water,7.6,76.0,92.0,77.0,James Cameron,https://watch.plex.tv/movie/avatar-2
3,Titanic,6.1,60.0,57.0,62.0,"Werner Klingler, Herbert Selpin",https://watch.plex.tv/movie/titanic
4,Star Wars: Episode VII - The Force Awakens,7.8,93.0,85.0,73.0,J.J. Abrams,https://watch.plex.tv/movie/star-wars-episode-...
...,...,...,...,...,...,...,...
995,The Scorpion King,5.5,41.0,38.0,56.0,Chuck Russell,https://watch.plex.tv/movie/the-scorpion-king
996,Sweet Home Alabama,6.2,38.0,78.0,64.0,Andy Tennant,https://watch.plex.tv/movie/sweet-home-alabama
997,Daddy's Home 2,6.0,21.0,51.0,63.0,Sean Anders,https://watch.plex.tv/movie/daddys-home-2
998,Hacksaw Ridge,8.1,84.0,91.0,82.0,Mel Gibson,https://watch.plex.tv/movie/hacksaw-ridge


In [1284]:
Mydf[Mydf["DomesticOpening"].str.contains("C")] # Columns contains some string values
columnstoconvert = ["Domestic Money","international","wordlwide"]
# Deleting $ Sign from3 columns

M1=Mydf["DomesticOpening"]
M1=list(M1)
for a in M1:
     a=str(a)
M1
for i,a in enumerate(M1):
    if "$" in a:
        M1[i]=(M1[i].replace('$', ''))
        M1[i]=int(M1[i].replace(",",""))
    else:
        M1[i]=np.nan
M1

Mydf["DomesticOpening"]=M1
Mydf

,ID,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,tt0499549,Avatar,2009,26.9%,73.1%,785221649,2138484377,2923706026,Twentieth Century FoxSee full company information,77025481.0,"$237,000,000","December 16, 2009 (EMEA, LATAM)",PG-13,2 hr 42 min,Action Adventure Fantasy...
1,tt4154796,Avengers: Endgame,2019,30.7%,69.3%,858373000,1941066100,2799439100,Walt Disney Studios Motion PicturesSee full co...,357115007.0,"$356,000,000","April 24, 2019 (21 markets)",PG-13,3 hr 1 min,Action Adventure Drama ...
2,tt1630029,Avatar: The Way of Water,2022,29.5%,70.5%,684075767,1636174514,2320250281,20th Century StudiosSee full company information,134100226.0,Unknown,"December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...
3,tt0120338,Titanic,1997,29.8%,70.2%,674292608,1590450697,2264743305,Paramount PicturesSee full company information,28638131.0,"$200,000,000","December 19, 1997 (Domestic)",PG-13,3 hr 14 min,Drama Romance
4,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,936662225,1134647993,2071310218,Walt Disney Studios Motion PicturesSee full co...,247966675.0,"$245,000,000","December 16, 2015 (APAC, EMEA)",PG-13,2 hr 18 min,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,2002,50.4%,49.6%,91047077,89583830,180630907,Universal PicturesSee full company information,36075875.0,"$60,000,000","April 18, 2002 (APAC)",PG-13,1 hr 32 min,Action Adventure Fantasy
996,tt0256415,Sweet Home Alabama,2002,70.4%,29.6%,127223418,53399006,180622424,Walt Disney Studios Motion PicturesSee full co...,35648740.0,"$30,000,000","September 27, 2002 (Domestic)",PG-13,1 hr 48 min,Comedy Romance
997,tt5657846,Daddy's Home 2,2017,57.6%,42.4%,104029443,76584381,180613824,Paramount PicturesSee full company information,29651193.0,"$69,000,000","November 9, 2017 (Netherlands)",PG-13,1 hr 40 min,Comedy
998,tt2119532,Hacksaw Ridge,2016,37.2%,62.8%,67209615,113354021,180563636,Lions Gate FilmsSee full company information,15190758.0,"$40,000,000","November 3, 2016 (EMEA, APAC)",R,2 hr 19 min,Biography Drama History ...


In [1285]:
M1=Mydf["Budget"]
M1=list(M1)
for a in M1:
    a=str(a)
for i,a in enumerate(M1):
    if "$" in a:
        M1[i]=(M1[i].replace('$', ''))
        M1[i]=int(M1[i].replace(",",""))
    else:
          M1[i]=np.nan
Mydf["Budget"]=M1
Mydf

,ID,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,tt0499549,Avatar,2009,26.9%,73.1%,785221649,2138484377,2923706026,Twentieth Century FoxSee full company information,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,2 hr 42 min,Action Adventure Fantasy...
1,tt4154796,Avengers: Endgame,2019,30.7%,69.3%,858373000,1941066100,2799439100,Walt Disney Studios Motion PicturesSee full co...,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,3 hr 1 min,Action Adventure Drama ...
2,tt1630029,Avatar: The Way of Water,2022,29.5%,70.5%,684075767,1636174514,2320250281,20th Century StudiosSee full company information,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...
3,tt0120338,Titanic,1997,29.8%,70.2%,674292608,1590450697,2264743305,Paramount PicturesSee full company information,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,3 hr 14 min,Drama Romance
4,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,936662225,1134647993,2071310218,Walt Disney Studios Motion PicturesSee full co...,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,2 hr 18 min,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,2002,50.4%,49.6%,91047077,89583830,180630907,Universal PicturesSee full company information,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,1 hr 32 min,Action Adventure Fantasy
996,tt0256415,Sweet Home Alabama,2002,70.4%,29.6%,127223418,53399006,180622424,Walt Disney Studios Motion PicturesSee full co...,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,1 hr 48 min,Comedy Romance
997,tt5657846,Daddy's Home 2,2017,57.6%,42.4%,104029443,76584381,180613824,Paramount PicturesSee full company information,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,1 hr 40 min,Comedy
998,tt2119532,Hacksaw Ridge,2016,37.2%,62.8%,67209615,113354021,180563636,Lions Gate FilmsSee full company information,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,2 hr 19 min,Biography Drama History ...


In [1291]:
Mydf.isna().sum() #NUMERIC COLUMNS READY FOR FURTHER ANALYSIS

ID                       0
Name                     0
Year                     0
Domestic Percentage      0
Foreign percentage       0
Domestic Money           0
international            0
wordlwide                0
distributor              0
DomesticOpening          9
Budget                 221
Date                     0
MPAA                     0
Time                     0
genres                   0
dtype: int64

In [1293]:
Mydf

,ID,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,tt0499549,Avatar,2009,26.9%,73.1%,785221649,2138484377,2923706026,Twentieth Century FoxSee full company information,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,2 hr 42 min,Action Adventure Fantasy...
1,tt4154796,Avengers: Endgame,2019,30.7%,69.3%,858373000,1941066100,2799439100,Walt Disney Studios Motion PicturesSee full co...,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,3 hr 1 min,Action Adventure Drama ...
2,tt1630029,Avatar: The Way of Water,2022,29.5%,70.5%,684075767,1636174514,2320250281,20th Century StudiosSee full company information,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...
3,tt0120338,Titanic,1997,29.8%,70.2%,674292608,1590450697,2264743305,Paramount PicturesSee full company information,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,3 hr 14 min,Drama Romance
4,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,936662225,1134647993,2071310218,Walt Disney Studios Motion PicturesSee full co...,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,2 hr 18 min,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,2002,50.4%,49.6%,91047077,89583830,180630907,Universal PicturesSee full company information,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,1 hr 32 min,Action Adventure Fantasy
996,tt0256415,Sweet Home Alabama,2002,70.4%,29.6%,127223418,53399006,180622424,Walt Disney Studios Motion PicturesSee full co...,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,1 hr 48 min,Comedy Romance
997,tt5657846,Daddy's Home 2,2017,57.6%,42.4%,104029443,76584381,180613824,Paramount PicturesSee full company information,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,1 hr 40 min,Comedy
998,tt2119532,Hacksaw Ridge,2016,37.2%,62.8%,67209615,113354021,180563636,Lions Gate FilmsSee full company information,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,2 hr 19 min,Biography Drama History ...


In [1295]:
Mydf["distributor"]=Mydf.loc[:,"distributor"].str.replace("See full company information","")
Mydf

,ID,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,tt0499549,Avatar,2009,26.9%,73.1%,785221649,2138484377,2923706026,Twentieth Century Fox,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,2 hr 42 min,Action Adventure Fantasy...
1,tt4154796,Avengers: Endgame,2019,30.7%,69.3%,858373000,1941066100,2799439100,Walt Disney Studios Motion Pictures,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,3 hr 1 min,Action Adventure Drama ...
2,tt1630029,Avatar: The Way of Water,2022,29.5%,70.5%,684075767,1636174514,2320250281,20th Century Studios,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,3 hr 12 min,Action Adventure Fantasy...
3,tt0120338,Titanic,1997,29.8%,70.2%,674292608,1590450697,2264743305,Paramount Pictures,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,3 hr 14 min,Drama Romance
4,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,936662225,1134647993,2071310218,Walt Disney Studios Motion Pictures,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,2 hr 18 min,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,2002,50.4%,49.6%,91047077,89583830,180630907,Universal Pictures,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,1 hr 32 min,Action Adventure Fantasy
996,tt0256415,Sweet Home Alabama,2002,70.4%,29.6%,127223418,53399006,180622424,Walt Disney Studios Motion Pictures,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,1 hr 48 min,Comedy Romance
997,tt5657846,Daddy's Home 2,2017,57.6%,42.4%,104029443,76584381,180613824,Paramount Pictures,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,1 hr 40 min,Comedy
998,tt2119532,Hacksaw Ridge,2016,37.2%,62.8%,67209615,113354021,180563636,Lions Gate Films,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,2 hr 19 min,Biography Drama History ...


In [1355]:
#Converting Hour & minutes to only minutes for further calculations
Hours=Mydf.loc[:,"Time"].str.replace("min","").str.split()
Timeinmins=[]
for a in range(len(Hours)):
    if len(Hours[a])==3:
        Timeinmins.append(int(Hours[a][0])*60+int(Hours[a][2]))
    elif len(Hours[a])==2:
        Timeinmins.append(int(Hours[a][0])*60)
    else:
        Timeinmins.append(np.nan)
Timeinmins=Hours*60+Minutes
Timeinmins

In [1352]:
Timeinmins = pd.DataFrame(Timeinmins, columns=['Timeinmins'])
Hours=list(Hours)
Hours = pd.DataFrame(Hours, columns=['Hours',"HR","mins"])
result = pd.concat([Timeinmins, Hours], axis=1)
len(result["Timeinmins"][result["Timeinmins"].isna()])==len(result[result["Hours"]=="Unknown"])
#It means we got nans from only unknown values. In other words we didnt lose any valuable data.

True

In [1359]:
Mydf["Time"]=Timeinmins
#We changed all time data to minutes 
Mydf

,ID,Name,Year,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,distributor,DomesticOpening,Budget,Date,MPAA,Time,genres
0,tt0499549,Avatar,2009,26.9%,73.1%,785221649,2138484377,2923706026,Twentieth Century Fox,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,162.0,Action Adventure Fantasy...
1,tt4154796,Avengers: Endgame,2019,30.7%,69.3%,858373000,1941066100,2799439100,Walt Disney Studios Motion Pictures,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,181.0,Action Adventure Drama ...
2,tt1630029,Avatar: The Way of Water,2022,29.5%,70.5%,684075767,1636174514,2320250281,20th Century Studios,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,192.0,Action Adventure Fantasy...
3,tt0120338,Titanic,1997,29.8%,70.2%,674292608,1590450697,2264743305,Paramount Pictures,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,194.0,Drama Romance
4,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,45.2%,54.8%,936662225,1134647993,2071310218,Walt Disney Studios Motion Pictures,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,138.0,Action Adventure Sci-Fi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,2002,50.4%,49.6%,91047077,89583830,180630907,Universal Pictures,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,92.0,Action Adventure Fantasy
996,tt0256415,Sweet Home Alabama,2002,70.4%,29.6%,127223418,53399006,180622424,Walt Disney Studios Motion Pictures,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,108.0,Comedy Romance
997,tt5657846,Daddy's Home 2,2017,57.6%,42.4%,104029443,76584381,180613824,Paramount Pictures,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,100.0,Comedy
998,tt2119532,Hacksaw Ridge,2016,37.2%,62.8%,67209615,113354021,180563636,Lions Gate Films,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,139.0,Biography Drama History ...


In [1367]:
newdf=newdf.drop("All ranks",axis=1)
Myranks=newdf.drop("Name",axis=1)
Myranks

,Director,linkselection,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB
0,James Cameron,https://watch.plex.tv/movie/avatar,7.9,81,82,76
1,"Anthony Russo, Joe Russo",https://watch.plex.tv/movie/avengers-endgame,8.4,94,90,83
2,James Cameron,https://watch.plex.tv/movie/avatar-2,7.6,76,92,77
3,"Werner Klingler, Herbert Selpin",https://watch.plex.tv/movie/titanic,6.1,60,57,62
4,J.J. Abrams,https://watch.plex.tv/movie/star-wars-episode-...,7.8,93,85,73
...,...,...,...,...,...,...
995,Chuck Russell,https://watch.plex.tv/movie/the-scorpion-king,5.5,41,38,56
996,Andy Tennant,https://watch.plex.tv/movie/sweet-home-alabama,6.2,38,78,64
997,Sean Anders,https://watch.plex.tv/movie/daddys-home-2,6.0,21,51,63
998,Mel Gibson,https://watch.plex.tv/movie/hacksaw-ridge,8.1,84,91,82


In [1410]:
Movies['Domestic Percentage']=Movies['Domestic Percentage'].str.replace("%","")
Movies['Foreign percentage']=Movies['Foreign percentage'].str.replace("%","")
Movies['Domestic Percentage'] = Movies['Domestic Percentage'].replace("–", np.nan)
Movies['Foreign percentage'] = Movies['Foreign percentage'].replace("–", np.nan)
Movies['Domestic Percentage']=Movies['Domestic Percentage'].astype(float)
Movies['Foreign percentage']=Movies['Foreign percentage'].astype(float)
Movies

,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,Foreign percentage,Domestic Money,international,wordlwide,DomesticOpening,Budget,Date,MPAA,genres,linkselection
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81,82,76,...,73.1,785221649,2138484377,2923706026,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94,90,83,...,69.3,858373000,1941066100,2799439100,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76,92,77,...,70.5,684075767,1636174514,2320250281,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60,57,62,...,70.2,674292608,1590450697,2264743305,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,Drama Romance,https://watch.plex.tv/movie/titanic
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93,85,73,...,54.8,936662225,1134647993,2071310218,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41,38,56,...,49.6,91047077,89583830,180630907,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38,78,64,...,29.6,127223418,53399006,180622424,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21,51,63,...,42.4,104029443,76584381,180613824,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84,91,82,...,62.8,67209615,113354021,180563636,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge


In [1411]:
Movies.loc[:, "IMDB":"TMDB"] = Movies.loc[:, "IMDB":"TMDB"].astype(float)
Movies

/var/folders/bv/bgrpp52x05nf96zx_01sm2_40000gn/T/ipykernel_26775/1963218016.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  Movies.loc[:, "IMDB":"TMDB"] = Movies.loc[:, "IMDB":"TMDB"].astype(float)


,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,Foreign percentage,Domestic Money,international,wordlwide,DomesticOpening,Budget,Date,MPAA,genres,linkselection
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81.0,82.0,76.0,...,73.1,785221649,2138484377,2923706026,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94.0,90.0,83.0,...,69.3,858373000,1941066100,2799439100,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76.0,92.0,77.0,...,70.5,684075767,1636174514,2320250281,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60.0,57.0,62.0,...,70.2,674292608,1590450697,2264743305,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,Drama Romance,https://watch.plex.tv/movie/titanic
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93.0,85.0,73.0,...,54.8,936662225,1134647993,2071310218,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41.0,38.0,56.0,...,49.6,91047077,89583830,180630907,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38.0,78.0,64.0,...,29.6,127223418,53399006,180622424,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21.0,51.0,63.0,...,42.4,104029443,76584381,180613824,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84.0,91.0,82.0,...,62.8,67209615,113354021,180563636,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge


# 3.0 Stacking Final Dataframes 

In [1384]:
Movies3=pd.concat([Mydf,Myranks],axis=1)
Movies=Movies3.iloc[:,[0,1,15,8,13,2,17,18,19,20,3,4,5,6,7,9,10,11,12,14,16]]
Movies.columns

Index(['ID', 'Name', 'Director', 'distributor', 'Time', 'Year', 'IMDB',
       'Tomatometer', 'Rotten Tomatos Audience', 'TMDB', 'Domestic Percentage',
       'Foreign percentage', 'Domestic Money', 'international', 'wordlwide',
       'DomesticOpening', 'Budget', 'Date', 'MPAA', 'genres', 'linkselection'],
      dtype='object')

In [1414]:
Movies.to_excel("fixinglinks.xlsx")

In [1417]:
Movies # DataFrame is ready , Cleaned and Processed.

,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,Foreign percentage,Domestic Money,international,wordlwide,DomesticOpening,Budget,Date,MPAA,genres,linkselection
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81.0,82.0,76.0,...,73.1,785221649,2138484377,2923706026,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94.0,90.0,83.0,...,69.3,858373000,1941066100,2799439100,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76.0,92.0,77.0,...,70.5,684075767,1636174514,2320250281,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60.0,57.0,62.0,...,70.2,674292608,1590450697,2264743305,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,Drama Romance,https://watch.plex.tv/movie/titanic
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93.0,85.0,73.0,...,54.8,936662225,1134647993,2071310218,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41.0,38.0,56.0,...,49.6,91047077,89583830,180630907,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38.0,78.0,64.0,...,29.6,127223418,53399006,180622424,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21.0,51.0,63.0,...,42.4,104029443,76584381,180613824,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84.0,91.0,82.0,...,62.8,67209615,113354021,180563636,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge


In [1419]:
MovieIDs2

0      tt0499549
1      tt4154796
2      tt1630029
3      tt0120338
4      tt2488496
         ...    
995    tt0277296
996    tt0256415
997    tt5657846
998    tt2119532
999    tt0453467
Length: 1000, dtype: object

# 4.0 Scraping more data from website 3 to have more dimensions for our model.

In [1675]:
MovieID3=[] # Preparing for new scraping
for a in MovieIDs2:
     MovieID3.append("https://www.boxofficemojo.com/title/"+a+"/")
MovieID3[:10]

['https://www.boxofficemojo.com/title/tt0499549/',
 'https://www.boxofficemojo.com/title/tt4154796/',
 'https://www.boxofficemojo.com/title/tt1630029/',
 'https://www.boxofficemojo.com/title/tt0120338/',
 'https://www.boxofficemojo.com/title/tt2488496/',
 'https://www.boxofficemojo.com/title/tt4154756/',
 'https://www.boxofficemojo.com/title/tt10872600/',
 'https://www.boxofficemojo.com/title/tt0369610/',
 'https://www.boxofficemojo.com/title/tt6105098/',
 'https://www.boxofficemojo.com/title/tt0848228/']

In [1529]:
#Added all Movie stars. We already scrape data , error is related to Indexing.
max_retries = 3  
retry_delay = 5
R=requests.get("https://www.imdb.com/list/ls000004615/?sort=list_order,asc&mode=detail&page=3")
Soup=BeautifulSoup(R.text,"html.parser")
for a in tqdm(range(52)):
    for _ in range(max_retries):
        try:
            Cast=Soup.find_all("h3",{"class":"lister-item-header"})[a].text
            Movieman.append(Cast)
            break
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)

 98%|█████████████████████████████████████████▏| 51/52 [00:00<00:00, 933.91it/s]


IndexError: list index out of range

In [1530]:
len(Movieman)

251

In [1434]:
import time 
myhref = []

max_retries = 3  # Maximum number of retries
retry_delay = 5  # Delay between retries in seconds

for a in tqdm(MovieID3):
    for _ in range(max_retries):
        try:
            R = requests.get(a, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser")
            anchor_elements = Soup.find_all('a')

            for a in anchor_elements:
                href = a.get('href')
                if "/credits/?ref_=bo_tt_tab#tabs" in href:
                    myhref.append(href)

            break  # If the request was successful, exit the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)

  1%|▎                                         | 6/1000 [00:11<18:12,  1.10s/it]

Request error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


  3%|█▎                                       | 32/1000 [00:34<09:37,  1.68it/s]

Request error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


  5%|██                                       | 50/1000 [00:54<13:05,  1.21it/s]

...

 95%|█████████████████████████████████████▉  | 948/1000 [16:24<00:24,  2.10it/s]

Request error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


 99%|███████████████████████████████████████▋| 992/1000 [16:57<00:05,  1.59it/s]

Request error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|███████████████████████████████████████| 1000/1000 [17:08<00:00,  1.03s/it]


In [1676]:
for i,a in enumerate(myhref):
    myhref[i]="https://www.boxofficemojo.com/"+a
myhref[:10]

['https://www.boxofficemojo.com//title/tt0499549/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt4154796/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt1630029/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt0120338/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt2488496/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt4154756/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt10872600/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt0369610/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt6105098/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com//title/tt0848228/credits/?ref_=bo_tt_tab#tabs']

In [1488]:
max_retries = 3
retry_delay = 6

for a in tqdm(myhref2):
    for _ in range(max_retries):
        try:
            R = requests.get(a)
            Soup = BeautifulSoup(R.text, "html.parser")
            anchor_elements = Soup.find_all('tr')
            last_8_elements_text = [element.get_text() for element in anchor_elements[-8:]]
            allcast.append(last_8_elements_text)
            break  # If the request was successful, exit the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            # Wait for the retry_delay before trying again
            time.sleep(retry_delay)

100%|█████████████████████████████████████████| 163/163 [01:48<00:00,  1.50it/s]


In [1490]:
len(allcast)
allcast2=allcast.copy()

# 4.1 Cleaning & Processing latest Dataframe for Machine learning

In [1494]:
allcast=pd.DataFrame(allcast)
allcast # We scraped more than actors , So we need some cleaning

,0,1,2,3,4,5,6,7
0,Stephen E. Rivkin\n\nEditor,Rick Carter\n\nProduction Designer,Robert Stromberg\n\nProduction Designer,CastRole,Sam Worthington\n\nJake SullySee more,Zoe Saldana\n\nNeytiriSee more,Sigourney Weaver\n\nDr. Grace AugustineSee more,Michelle Rodriguez\n\nTrudy ChacónSee more
1,Jeffrey Ford\n\nEditor,Matthew Schmidt\n\nEditor,Charles Wood\n\nProduction Designer,CastRole,Robert Downey Jr.\n\nTony Stark/Iron ManSee more,Chris Evans\n\nSteve Rogers/Captain AmericaSee...,Mark Ruffalo\n\nBruce Banner/HulkSee more,Chris Hemsworth\n\nThorSee more
2,Stephen E. Rivkin\n\nEditor,Dylan Cole\n\nProduction Designer,Ben Procter\n\nProduction Designer,CastRole,Sam Worthington\n\nJakeSee more,Zoe Saldana\n\nNeytiriSee more,Sigourney Weaver\n\nKiriSee more,Stephen Lang\n\nQuaritchSee more
3,James Cameron\n\nEditor,Richard A. Harris\n\nEditor,Peter Lamont\n\nProduction Designer,CastRole,Leonardo DiCaprio\n\nJack DawsonSee more,Kate Winslet\n\nRose Dewitt BukaterSee more,Billy Zane\n\nCal HockleySee more,Kathy Bates\n\nMolly BrownSee more
4,Mary Jo Markey\n\nEditor,Rick Carter\n\nProduction Designer,Darren Gilford\n\nProduction Designer,CastRole,Daisy Ridley\n\nReySee more,John Boyega\n\nFinnSee more,Oscar Isaac\n\nPoe DameronSee more,Domhnall Gleeson\n\nGeneral HuxSee more
...,...,...,...,...,...,...,...,...
995,Greg Parsons\n\nEditor,Michael Tronick\n\nEditor,Ed Verreaux\n\nProduction Designer,CastRole,Dwayne Johnson\n\nMathayusSee more,Steven Brand\n\nMemnonSee more,Michael Clarke Duncan\n\nBalthazarSee more,Kelly Hu\n\nThe SorceressSee more
996,Troy Takaki\n\nEditor,Tracey Wadmore-Smith\n\nEditor,Clay A. Griffith\n\nProduction Designer,CastRole,Reese Witherspoon\n\nMelanie SmooterSee more,Patrick Dempsey\n\nAndrew HenningsSee more,Josh Lucas\n\nJake PerrySee more,Candice Bergen\n\nMayor Kate HenningsSee more
997,Julio Macat\n\nCinematographer,Brad Wilhite\n\nEditor,Clayton Hartley\n\nProduction Designer,CastRole,Will Ferrell\n\nBradSee more,Mark Wahlberg\n\nDustySee more,Mel Gibson\n\nKurtSee more,John Lithgow\n\nDonSee more
998,Simon Duggan\n\nCinematographer,John Gilbert\n\nEditor,Barry Robison\n\nProduction Designer,CastRole,Andrew Garfield\n\nDesmond DossSee more,Sam Worthington\n\nCaptain GloverSee more,Luke Bracey\n\nSmitty RykerSee more,Teresa Palmer\n\nDorothy SchutteSee more


In [1497]:
allcast=allcast.drop(columns=[0,1,2,3])

In [1554]:
Moviess=pd.concat([Movies,allcast],axis=1)
Moviess

,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,DomesticOpening,Budget,Date,MPAA,genres,linkselection,4,5,6,7
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81.0,82.0,76.0,...,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar,Sam Worthington\n\nJake SullySee more,Zoe Saldana\n\nNeytiriSee more,Sigourney Weaver\n\nDr. Grace AugustineSee more,Michelle Rodriguez\n\nTrudy ChacónSee more
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94.0,90.0,83.0,...,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame,Robert Downey Jr.\n\nTony Stark/Iron ManSee more,Chris Evans\n\nSteve Rogers/Captain AmericaSee...,Mark Ruffalo\n\nBruce Banner/HulkSee more,Chris Hemsworth\n\nThorSee more
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76.0,92.0,77.0,...,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2,Sam Worthington\n\nJakeSee more,Zoe Saldana\n\nNeytiriSee more,Sigourney Weaver\n\nKiriSee more,Stephen Lang\n\nQuaritchSee more
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60.0,57.0,62.0,...,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,Drama Romance,https://watch.plex.tv/movie/titanic,Leonardo DiCaprio\n\nJack DawsonSee more,Kate Winslet\n\nRose Dewitt BukaterSee more,Billy Zane\n\nCal HockleySee more,Kathy Bates\n\nMolly BrownSee more
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93.0,85.0,73.0,...,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...,Daisy Ridley\n\nReySee more,John Boyega\n\nFinnSee more,Oscar Isaac\n\nPoe DameronSee more,Domhnall Gleeson\n\nGeneral HuxSee more
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41.0,38.0,56.0,...,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king,Dwayne Johnson\n\nMathayusSee more,Steven Brand\n\nMemnonSee more,Michael Clarke Duncan\n\nBalthazarSee more,Kelly Hu\n\nThe SorceressSee more
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38.0,78.0,64.0,...,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama,Reese Witherspoon\n\nMelanie SmooterSee more,Patrick Dempsey\n\nAndrew HenningsSee more,Josh Lucas\n\nJake PerrySee more,Candice Bergen\n\nMayor Kate HenningsSee more
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21.0,51.0,63.0,...,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2,Will Ferrell\n\nBradSee more,Mark Wahlberg\n\nDustySee more,Mel Gibson\n\nKurtSee more,John Lithgow\n\nDonSee more
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84.0,91.0,82.0,...,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge,Andrew Garfield\n\nDesmond DossSee more,Sam Worthington\n\nCaptain GloverSee more,Luke Bracey\n\nSmitty RykerSee more,Teresa Palmer\n\nDorothy SchutteSee more


In [1555]:
for a in range(4,8):
    split_values = Moviess[a].str.split("\n\n")
    actor_names = split_values.str[0]
    Moviess[a] = actor_names
Moviess

,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,DomesticOpening,Budget,Date,MPAA,genres,linkselection,4,5,6,7
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81.0,82.0,76.0,...,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar,Sam Worthington,Zoe Saldana,Sigourney Weaver,Michelle Rodriguez
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94.0,90.0,83.0,...,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76.0,92.0,77.0,...,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2,Sam Worthington,Zoe Saldana,Sigourney Weaver,Stephen Lang
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60.0,57.0,62.0,...,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,Drama Romance,https://watch.plex.tv/movie/titanic,Leonardo DiCaprio,Kate Winslet,Billy Zane,Kathy Bates
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93.0,85.0,73.0,...,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...,Daisy Ridley,John Boyega,Oscar Isaac,Domhnall Gleeson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41.0,38.0,56.0,...,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king,Dwayne Johnson,Steven Brand,Michael Clarke Duncan,Kelly Hu
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38.0,78.0,64.0,...,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama,Reese Witherspoon,Patrick Dempsey,Josh Lucas,Candice Bergen
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21.0,51.0,63.0,...,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2,Will Ferrell,Mark Wahlberg,Mel Gibson,John Lithgow
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84.0,91.0,82.0,...,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge,Andrew Garfield,Sam Worthington,Luke Bracey,Teresa Palmer


In [1562]:
#Combining all actors in One column
Moviess['Actors'] = Moviess[4].astype(str) + "," + Moviess[5].astype(str) + "," + Moviess[6].astype(str)+ "," + Moviess[7].astype(str)
Moviess.drop([4, 5, 6, 7], axis=1, inplace=True)
Moviess

,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,Domestic Money,international,wordlwide,DomesticOpening,Budget,Date,MPAA,genres,linkselection,Actors
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81.0,82.0,76.0,...,785221649,2138484377,2923706026,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar,"Sam Worthington,Zoe Saldana,Sigourney Weaver,M..."
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94.0,90.0,83.0,...,858373000,1941066100,2799439100,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame,"Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chr..."
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76.0,92.0,77.0,...,684075767,1636174514,2320250281,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2,"Sam Worthington,Zoe Saldana,Sigourney Weaver,S..."
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60.0,57.0,62.0,...,674292608,1590450697,2264743305,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,Drama Romance,https://watch.plex.tv/movie/titanic,"Leonardo DiCaprio,Kate Winslet,Billy Zane,Kath..."
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93.0,85.0,73.0,...,936662225,1134647993,2071310218,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...,"Daisy Ridley,John Boyega,Oscar Isaac,Domhnall ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41.0,38.0,56.0,...,91047077,89583830,180630907,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king,"Dwayne Johnson,Steven Brand,Michael Clarke Dun..."
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38.0,78.0,64.0,...,127223418,53399006,180622424,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama,"Reese Witherspoon,Patrick Dempsey,Josh Lucas,C..."
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21.0,51.0,63.0,...,104029443,76584381,180613824,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2,"Will Ferrell,Mark Wahlberg,Mel Gibson,John Lit..."
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84.0,91.0,82.0,...,67209615,113354021,180563636,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge,"Andrew Garfield,Sam Worthington,Luke Bracey,Te..."


In [1677]:
for i, a in enumerate(Movieman):
    Movieman[i] = a.split("\n")[2]
Movieman[:10]

['JackNicholson',
 'RalphFiennes',
 'DanielDay-Lewis',
 'RobertDeNiro',
 'AlPacino',
 'DustinHoffman',
 'TomHanks',
 'BradPitt',
 'AnthonyHopkins',
 'MarlonBrando']

In [1565]:
Movies=Moviess.copy()
Movies

,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,Domestic Money,international,wordlwide,DomesticOpening,Budget,Date,MPAA,genres,linkselection,Actors
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81.0,82.0,76.0,...,785221649,2138484377,2923706026,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar,"Sam Worthington,Zoe Saldana,Sigourney Weaver,M..."
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94.0,90.0,83.0,...,858373000,1941066100,2799439100,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame,"Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chr..."
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76.0,92.0,77.0,...,684075767,1636174514,2320250281,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2,"Sam Worthington,Zoe Saldana,Sigourney Weaver,S..."
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60.0,57.0,62.0,...,674292608,1590450697,2264743305,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,Drama Romance,https://watch.plex.tv/movie/titanic,"Leonardo DiCaprio,Kate Winslet,Billy Zane,Kath..."
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93.0,85.0,73.0,...,936662225,1134647993,2071310218,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...,"Daisy Ridley,John Boyega,Oscar Isaac,Domhnall ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41.0,38.0,56.0,...,91047077,89583830,180630907,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king,"Dwayne Johnson,Steven Brand,Michael Clarke Dun..."
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38.0,78.0,64.0,...,127223418,53399006,180622424,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama,"Reese Witherspoon,Patrick Dempsey,Josh Lucas,C..."
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21.0,51.0,63.0,...,104029443,76584381,180613824,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2,"Will Ferrell,Mark Wahlberg,Mel Gibson,John Lit..."
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84.0,91.0,82.0,...,67209615,113354021,180563636,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge,"Andrew Garfield,Sam Worthington,Luke Bracey,Te..."


In [1678]:
for i, a in enumerate(Movieman):
    Movieman[i] = a.replace(" ", "")
Movieman[15:25]

['HenryFonda',
 'WilliamHurt',
 'KevinCostner',
 'ClintEastwood',
 'LeonardoDiCaprio',
 'MelGibson',
 'RobertDuvall',
 'SamuelL.Jackson',
 'TommyLeeJones',
 'KevinSpacey']

In [1601]:
#Created a column that contains how many top 250 popular stars are performing in that movie.
for a in tqdm(Movieman):
    mask = Moviess["Actors"].str.contains(a, case=False)
    Moviess.loc[mask, "BestActorPresent"] += 1
Moviess 

100%|███████████████████████████████████████| 251/251 [00:00<00:00, 1344.95it/s]


,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,international,wordlwide,DomesticOpening,Budget,Date,MPAA,genres,linkselection,Actors,BestActorPresent
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81.0,82.0,76.0,...,2138484377,2923706026,77025481.0,237000000.0,"December 16, 2009 (EMEA, LATAM)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar,"SamWorthington,ZoeSaldana,SigourneyWeaver,Mich...",0
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94.0,90.0,83.0,...,1941066100,2799439100,357115007.0,356000000.0,"April 24, 2019 (21 markets)",PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame,"RobertDowneyJr.,ChrisEvans,MarkRuffalo,ChrisHe...",3
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76.0,92.0,77.0,...,1636174514,2320250281,134100226.0,NaN,"December 14, 2022 (APAC, EMEA)",PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2,"SamWorthington,ZoeSaldana,SigourneyWeaver,Step...",0
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60.0,57.0,62.0,...,1590450697,2264743305,28638131.0,200000000.0,"December 19, 1997 (Domestic)",PG-13,Drama Romance,https://watch.plex.tv/movie/titanic,"LeonardoDiCaprio,KateWinslet,BillyZane,KathyBates",2
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93.0,85.0,73.0,...,1134647993,2071310218,247966675.0,245000000.0,"December 16, 2015 (APAC, EMEA)",PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...,"DaisyRidley,JohnBoyega,OscarIsaac,DomhnallGleeson",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41.0,38.0,56.0,...,89583830,180630907,36075875.0,60000000.0,"April 18, 2002 (APAC)",PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king,"DwayneJohnson,StevenBrand,MichaelClarkeDuncan,...",2
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38.0,78.0,64.0,...,53399006,180622424,35648740.0,30000000.0,"September 27, 2002 (Domestic)",PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama,"ReeseWitherspoon,PatrickDempsey,JoshLucas,Cand...",0
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21.0,51.0,63.0,...,76584381,180613824,29651193.0,69000000.0,"November 9, 2017 (Netherlands)",PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2,"WillFerrell,MarkWahlberg,MelGibson,JohnLithgow",2
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84.0,91.0,82.0,...,113354021,180563636,15190758.0,40000000.0,"November 3, 2016 (EMEA, APAC)",R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge,"AndrewGarfield,SamWorthington,LukeBracey,Teres...",0


In [1602]:
Moviess.groupby(["BestActorPresent"]).sum()

/var/folders/bv/bgrpp52x05nf96zx_01sm2_40000gn/T/ipykernel_26775/1613087467.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Moviess.groupby(["BestActorPresent"]).sum()


,Time,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,Domestic Percentage,Foreign percentage,Domestic Money,international,wordlwide,DomesticOpening,Budget
BestActorPresent,,,,,,,,,,,,
0,41474.0,2723.6,22991.0,25086.0,24441.0,14911.1,26260.6,67869173652,114084468072,170653082437,1.788147e+10,2.612158e+10
2,36565.0,2212.0,19584.0,21542.0,21200.0,13223.3,19242.9,52165138654,81573934243,133952080199,1.286092e+10,2.708860e+10
3,22979.0,1301.2,11991.0,12565.0,12416.0,7706.6,11093.4,31795408679,48880684633,80676093312,8.098587e+09,1.610010e+10
4,7987.0,444.9,4010.0,4357.0,4171.0,2614.9,3585.0,10352708414,15329187894,25682218738,2.662859e+09,5.626000e+09
5,1918.0,101.0,898.0,992.0,916.0,491.2,808.8,2494824438,4240232579,6735057017,5.871486e+08,1.006000e+09


In [1603]:
#having multiple dummy columns for machine learning
Moviess['ActorCategory'] = 'NOactor'
Moviess.loc[Moviess['BestActorPresent'] == 2, 'ActorCategory'] = 'OneActor'
Moviess.loc[Moviess['BestActorPresent'] == 3, 'ActorCategory'] = 'Twoactors'
Moviess.loc[Moviess['BestActorPresent'] == 4, 'ActorCategory'] = 'Threeactors'
Moviess.loc[Moviess['BestActorPresent'] >= 5, 'ActorCategory'] = 'Fouractors'


binary_columns = pd.get_dummies(Moviess['ActorCategory'], drop_first=False)


Moviess = pd.concat([Moviess, binary_columns], axis=1)

Moviess = Moviess.drop('ActorCategory', axis=1)
Moviess

,ID,Name,Director,distributor,Time,Year,IMDB,Tomatometer,Rotten Tomatos Audience,TMDB,...,MPAA,genres,linkselection,Actors,BestActorPresent,Fouractors,NOactor,OneActor,Threeactors,Twoactors
0,tt0499549,Avatar,James Cameron,Twentieth Century Fox,162.0,2009,7.9,81.0,82.0,76.0,...,PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar,"SamWorthington,ZoeSaldana,SigourneyWeaver,Mich...",0,0,1,0,0,0
1,tt4154796,Avengers: Endgame,"Anthony Russo, Joe Russo",Walt Disney Studios Motion Pictures,181.0,2019,8.4,94.0,90.0,83.0,...,PG-13,Action Adventure Drama ...,https://watch.plex.tv/movie/avengers-endgame,"RobertDowneyJr.,ChrisEvans,MarkRuffalo,ChrisHe...",3,0,0,0,0,1
2,tt1630029,Avatar: The Way of Water,James Cameron,20th Century Studios,192.0,2022,7.6,76.0,92.0,77.0,...,PG-13,Action Adventure Fantasy...,https://watch.plex.tv/movie/avatar-2,"SamWorthington,ZoeSaldana,SigourneyWeaver,Step...",0,0,1,0,0,0
3,tt0120338,Titanic,"Werner Klingler, Herbert Selpin",Paramount Pictures,194.0,1997,6.1,60.0,57.0,62.0,...,PG-13,Drama Romance,https://watch.plex.tv/movie/titanic,"LeonardoDiCaprio,KateWinslet,BillyZane,KathyBates",2,0,0,1,0,0
4,tt2488496,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,Walt Disney Studios Motion Pictures,138.0,2015,7.8,93.0,85.0,73.0,...,PG-13,Action Adventure Sci-Fi,https://watch.plex.tv/movie/star-wars-episode-...,"DaisyRidley,JohnBoyega,OscarIsaac,DomhnallGleeson",0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0277296,The Scorpion King,Chuck Russell,Universal Pictures,92.0,2002,5.5,41.0,38.0,56.0,...,PG-13,Action Adventure Fantasy,https://watch.plex.tv/movie/the-scorpion-king,"DwayneJohnson,StevenBrand,MichaelClarkeDuncan,...",2,0,0,1,0,0
996,tt0256415,Sweet Home Alabama,Andy Tennant,Walt Disney Studios Motion Pictures,108.0,2002,6.2,38.0,78.0,64.0,...,PG-13,Comedy Romance,https://watch.plex.tv/movie/sweet-home-alabama,"ReeseWitherspoon,PatrickDempsey,JoshLucas,Cand...",0,0,1,0,0,0
997,tt5657846,Daddy's Home 2,Sean Anders,Paramount Pictures,100.0,2017,6.0,21.0,51.0,63.0,...,PG-13,Comedy,https://watch.plex.tv/movie/daddys-home-2,"WillFerrell,MarkWahlberg,MelGibson,JohnLithgow",2,0,0,1,0,0
998,tt2119532,Hacksaw Ridge,Mel Gibson,Lions Gate Films,139.0,2016,8.1,84.0,91.0,82.0,...,R,Biography Drama History ...,https://watch.plex.tv/movie/hacksaw-ridge,"AndrewGarfield,SamWorthington,LukeBracey,Teres...",0,0,1,0,0,0


In [1604]:
Moviess.to_csv("Readytogo.csv") # We ready to go for some statistical analysis

# Storing Final Data In SQL Data Warehouse

In [1679]:
output100=Moviess.copy()

In [1680]:
host = "localhost"
user = "root"
password = "Ali85forme"
database = "Proje"  
try:
    mydb = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    if mydb.is_connected(): 
        table_name = 'Movio' 
        cursor = mydb.cursor()
        create_table_query = (
            f"CREATE TABLE IF NOT EXISTS {table_name} ("
            "ID INT AUTO_INCREMENT PRIMARY KEY,"
            "Name VARCHAR(255),"
            "Director VARCHAR(255),"
            "distributor VARCHAR(255),"
            "Time INT,"
            "Year INT,"
            "IMDB DECIMAL(5, 2),"
            "Tomatometer DECIMAL(5, 2),"
            "RottenTomatosAudience DECIMAL(5, 2),"
            "TMDB INT,"
            "DomesticPercentage DECIMAL(5, 2),"
            "Foreignpercentage DECIMAL(5, 2),"
            "DomesticMoney INT,"
            "international DECIMAL(15, 2),"
            "wordlwide DECIMAL(15, 2),"
            "DomesticOpening DECIMAL(15, 2),"
            "Budget DECIMAL(15, 2),"
            "Date2 VARCHAR(255),"
            "MPAA VARCHAR(10),"
            "genres VARCHAR(255),"
            "linkselection VARCHAR(255),"
            "Actors TEXT,"
            "BestActorPresent VARCHAR(255),"
            "Fouractors VARCHAR(255),"
            "NOactor VARCHAR(255),"
            "OneActor VARCHAR(255),"
            "Threeactors VARCHAR(255),"
            "Twoactors VARCHAR(255),"
            "GenreCluster VARCHAR(255)"
            ")"
        )
        cursor.execute(create_table_query)
        insert_query = (
            f"INSERT INTO {table_name} ("
            "Name, Director, distributor, Time, Year, IMDB, Tomatometer, RottenTomatosAudience, "
            "TMDB, DomesticPercentage, Foreignpercentage, DomesticMoney, international, wordlwide, "
            "DomesticOpening, Budget, Date2, MPAA, genres, linkselection, Actors, BestActorPresent, "
            "Fouractors, NOactor, OneActor, Threeactors, Twoactors, GenreCluster) "
            "VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        )
        for index, row in output100.iterrows():
            data = (
                row['Name'], row['Director'], row['distributor'], row['Time'], row['Year'],
                row['IMDB'], row['Tomatometer'], row['RottenTomatosAudience'], row['TMDB'],
                row['DomesticPercentage'], row['Foreignpercentage'], row['DomesticMoney'],
                row['international'], row['wordlwide'], row['DomesticOpening'], row['Budget'],
                row['Date2'], row['MPAA'], row['genres'], row['linkselection'], row['Actors'],
                row['BestActorPresent'], row['Fouractors'], row['NOactor'], row['OneActor'],
                row['Threeactors'], row['Twoactors'], row['GenreCluster']
            )
            cursor.execute(insert_query, data)
        mydb.commit()
        cursor.close()
        mydb.close()
except Error as e:
    print("Error:", e)

In [1681]:
del output100
"""Deleting DataFrame for freeing memory"""

'Deleting DataFrame for freeing memory'